# 装饰器 @staticmethod 和 @classmethod

https://blog.csdn.net/jypfhx/article/details/75045471

### 实例方法
通常，在python类中定义的普通方法就是实例方法。它的语法如下：

```
class C(object):
    def fun(self,arg1,arg2,...):
```
实例方法的参数中有一个必不可少的参数self，它必须位于参数列表的第一个位置，它相当于该方法所属的类在实例化后的实例本身(Instance)，表示传入该方法所属类的一个实例。实例方法被调用时，参数self被隐式地传入方法中，无需显式指定。

既然叫做实例方法，那么它必须使用该类的实例才能调用
`C().fun() `，直接使用类来调用该方法是无效的` C.fun() `。

### staticmethod（静态方法）
需要一个装饰器语法(@staticmethod)将一个普通方法（实例方法）转换为静态方法：

```
class C(object):
    @staticmethod
    def fun(arg1,arg2,...):
```
staticmethod 方法的参数是自定义的任何参数，当然也可以没有参数。它不需要一个隐含参数，像self这样的。这是它与实例方法和类方法之间在形式上的一个主要不同之处。

staticmethod即可以在类上调用`C.fun()`也可以在实例上调用`C().fun()`. **在实例上调用时，实例被忽略而类被使用---实际上还是类调用。**

staticmethod不需要自身对象self和自身类的cls参数，没有对类或实例的依赖，它处理与类无关的信息，就像恰好放在类中的一个局部函数，但是它的功能又与类有某种关联。

如果需要在staticmethod中调用类的属性和方法，可以直接用类名.属性名/类名.方法名。

### classmethod（类方法）
需要一个装饰器语法（@classmethod）将一个普通方法（实例方法）转换为类方法：

```
class C(object):
    @classmethod
    def fun(cls,arg1,arg2,...):
```
classmethod需要接收一个隐含参数作为第一个参数，不同于实例方法的self, 类方法的隐含参数叫做cls(其实可以叫任何别的名字，这里主要是因为命名惯例), 表示传入该方法所属的类。 

classmethod即可以在类上调用（C.f()）也可以在实例上调用（C().f()）. **在实例上调用时，实例被忽略而类被使用---实际上还是类调用。**

如果classmethod是在派生类中被调用的，被隐式传入cls参数的是那个子类对象，而不是父类。

### 应用举例
#### 基本使用
定义一个实例方法，静态方法和类方法，并分别进行调用。

In [2]:
class A(object):
    
    def insm(self):
        print('Instance method:',self)
        
    @staticmethod
    def stam():
        print('Static method')
        
    @classmethod
    def clsm(cls):
        print('Class method:',cls)
    

# 在实例上调用实例方法
A().insm()

# 在实例上调用静态方法
A().clsm()

# 在实例上调用类方法
A().stam()

# 在类上调用类方法
A.clsm()

# 在类上调用静态方法
A.stam()

# 在类上调用类实例方法 (报错)
A.insm()

Instance method: <__main__.A object at 0x001B6370>
Class method: <class '__main__.A'>
Static method
Class method: <class '__main__.A'>
Static method


TypeError: insm() missing 1 required positional argument: 'self'

####  在派生类中使用父类的类方法

如果classmethod是在派生类中被调用的，被隐式传入cls参数的是那个子类对象，而不是父类。

下面例子显示的cls参数值为子类对象

In [3]:
class B(A):
    pass

B.clsm()

Class method: <class '__main__.B'>


# Python中的staticmethod与classmethod

https://www.cnblogs.com/agnewee/p/5653936.html

# 动态给实例／class 绑定属性和方法

正常情况下，当我们定义了一个 class，创建了一个 class的实例后，我们可以**给该实例绑定任何属性和方法**，这就是**动态语言的灵活性**。

下面是给一个实例绑定属性和方法，**注意给某个实例绑定方法，对另一个实例是不起作用的**

为了给所有实例绑定方法，可以给 class绑定方法。

In [11]:
# 定义 class
class Student():
    pass

s = Student()

# 动态地给实例绑定属性
s.name = 'Jamie'
print(s.name)

# 动态地给实例绑定方法
def set_age(self,age): #定义一个函数作为实例的方法
    self.age = age 
    
from types import MethodType
s.set_age = MethodType(set_age,s) # 给实例绑定方法
s.set_age(28)
print(s.age)

# 该方法对另一个实例不起作用
s2 =Student()
s2.set_age(30) # 报错 AttributeError: 'student' object has no attribute 'set_age'

# 动态给class绑定方法
def set_score(self,score):
    self.score = score
    
Student.set_score = set_score
s.set_score(100)
s2.set_score (99)
print(s.score,s2.score)

Jamie
28
100 99


通常情况下，上面的set_score方法可以直接定义在class中，但动态绑定允许我们在程序运行的过程中动态给class加上功能，这在静态语言中很难实现。

# 使用\__slots__

当我们想要限制实例的属性怎么办？比如，只允许对Student实例添加name和age属性。为了达到限制的目的，Python允许在定义class的时候，**定义一个特殊的\__slots__变量，来限制该class实例能添加的属性**

由于'score'没有被放到\__slots__中，所以不能绑定score属性，试图绑定score将得到AttributeError的错误。

**注意\__slots__定义的属性仅对当前类实例起作用，对继承的子类是不起作用的**

除非在子类中也定义\__slots__，这样，**子类实例允许定义的属性就是自身的\__slots__加上父类的\__slots__**。

In [20]:
# 使用__slots__
class Student(object):
    __slots__ = ('name','age')
    
s = Student()
s.name = 'Jamie'
s.age = 28
# 由于'score'没有被放到__slots__中，所以不能绑定score属性，试图绑定score将得到AttributeError的错误。
#s.score = 100 # 报错 AttributeError: 'Student' object has no attribute 'score'


# slots定义的属性仅对当前类实例起作用，对继承的子类是不起作用的
class GraduateStudent(Student):
    pass
g = GraduateStudent()
g.score = 100
print(g.score)


#除非在子类中也定义__slots__，这样，子类实例允许定义的属性就是自身的__slots__加上父类的__slots__。

class Gstudent(Student):
    __slots__ = ('height')

gs = Gstudent()
gs.name = 'gs'
gs.age = 30
gs.height = 160
# gs.score =99 # AttributeError: 'Gstudent' object has no attribute 'score'

100


# 使用@property

为了限制score的范围，可以通过一个set_score()方法来设置成绩，再通过一个get_score()来获取成绩，这样，在set_score()方法里，就可以检查参数：

In [10]:
class Student(object):
    
    def get_score(self):
        return self._score
    
    def set_score(self,value):
        if not isinstance(value,int):
            raise ValueError('Score must be an interger!')
        if value < 0 or value > 100:
            raise ValueError('Score must be between 0~100')
            
        self._score = value
        

s = Student()
s.set_score(90)
s.get_score()
#s.set_score(999)

90

如果要让内部属性不被外部访问，可以把属性的名称前加上两个下划线\__，在Python中，实例的变量名如果以__开头，就变成了一个私有变量（private），只有内部可以访问，外部不能访问

有些时候，你会看到以一个下划线开头的实例变量名，比如_name，这样的实例变量外部是可以访问的，但是，按照约定俗成的规定，当你看到这样的变量时，意思就是，“虽然我可以被访问，但是，请把我视为私有变量，不要随意访问”。

但是，上面的调用方法又略显复杂，没有直接用属性这么直接简单。有没有既能检查参数，又可以用类似属性这样简单的方式来访问类的变量呢？还记得装饰器（decorator）可以给函数动态加上功能吗？对于类的方法，装饰器一样起作用。Python内置的@property装饰器就是负责把一个方法变成属性调用的：

In [18]:
class Student(object):
    
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self,value):
        if not isinstance(value,int):
            raise ValueError('Score must be an interger!')
        if value < 0 or value > 100:
            raise ValueError('Score must be between 0~100')
            
        self._score = value
        

s = Student()
s.score = 60 # 实际事转化s.set_score(60)
s.score # 实际事转化s.get_score()

# s.score = 999

60

@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作。 

注意到这个神奇的@property，我们在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现的。

还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性：

In [21]:
class Student(object):
    
    @property
    def birth(self):
        return self._birth
    
    @birth.setter
    def birth(self,value):
        self._birth = value
    
    @property
    def age(self):
        return 2018-self._birth
    
s = Student()
s.birth = 28
s.age

1990

上面的birth是可读写属性，而age就是一个只读属性，因为age可以根据birth和当前时间计算出来。

# 定制类

看到类似\__slots__这种形如\__xxx__的变量或者函数名就要注意，这些在Python中是有特殊用途的。

\__slots__我们已经知道怎么用了，\__len__()方法我们也知道是为了能让class作用于len()函数。

除此之外，Python的class中还有许多这样有特殊用途的函数，可以帮助我们定制类。

## \__str__

用print打印实例的结果： `<__main__.Student object at 0x1107b4588>`，不好看。怎么才能打印得好看呢？只需要定义好__str__()方法，返回一个好看的字符串就可以了：

In [25]:
class Student():
    def __init__(self,name):
        self.name = name

s = Student('Jamie')
print(s)

print('__str__ 使 print 打印出定制内容')

class Student():
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return 'Student object:{}'.format(self.name)

s = Student('Jamie')
print(s)

s

__str__ 使 print 打印出定制内容
Student object:Jamie


直接敲变量不用print，打印出来的实例还是不好看：`<__main__.Student at 0x110880f98>`

这是因为直接显示变量调用的不是\__str__()，而是\__repr__()，两者的区别是

- \__str__()返回用户看到的字符串

- \__repr__()返回程序开发者看到的字符串，也就是说，\__repr__()是为调试服务的。

解决办法是再定义一个\__repr__()。但是通常\__str__()和\__repr__()代码都是一样的，所以，有个偷懒的写法：

`__repr__ = __str__`

In [27]:
class Student():
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return 'Student object:{}'.format(self.name)
    __repr__ =  __str__ 
    
s = Student('Jamie')
s

Student object:Jamie

## \__getattr__

正常情况下，当我们调用类的方法或属性时，如果不存在，就会报错。

In [28]:
class Student(object):
    def __init__(self,name):
        self.name = name
        
s = Student('Jamie')
print(s.name)
print(s.score)

Jamie


AttributeError: 'Student' object has no attribute 'score'

要避免这个错误，除了可以加上一个score属性外，Python还有另一个机制，那就是写一个__getattr__()方法，动态返回一个属性。

In [36]:
class Student(object):
    def __init__(self,name):
        self.name = name
    
    def __getattr__(self,attr):
        if attr == 'score':
            return 99
        if attr=='age':
            return lambda: 25
        
s = Student('Jamie')
print(s.name)
print(s.score)
print(s.age())
print(s.abe)

Jamie
99
25
None


当调用不存在的属性时，比如score，Python解释器会试图调用__getattr__(self, 'score')来尝试获得属性，这样，我们就有机会返回score的值：

返回函数也是完全可以的,只是调用方式要在函数后面加().

注意，只有在没有找到属性的情况下，才调用__getattr__，已有的属性，比如name，不会在__getattr__中查找。

此外，注意到任意调用如s.abc都会返回None，这是因为我们定义的__getattr__默认返回就是None。要让class只响应特定的几个属性，我们就要按照约定，抛出AttributeError的错误：



# 两句话掌握python最难知识点——元类

https://segmentfault.com/a/1190000011447445


学懂元类，你只需要知道两句话：

### 道生一，一生二，二生三，三生万物

### 我是谁？我从哪来里？我要到哪里去？


在python世界，拥有一个永恒的道，那就是“type”，请记在脑海中，type就是道。如此广袤无垠的python生态圈，都是由type产生出来的。

道生一，一生二，二生三，三生万物。

道 即是 type

一 即是 metaclass(元类，或者叫类生成器)

二 即是 class(类，或者叫实例生成器)

三 即是 instance(实例)

万物 即是 实例的各种属性与方法，我们平常使用python时，调用的就是它们。


In [1]:
# 创建一个Hello类，拥有属性 say_heelo --- 二的起源
class Hello():
    def say_hello(self,name='world'):
        print('Hello,{}'.format(name))

# 从 Hello 类创建一个实例 hello  --- 二生三
hello = Hello()

# 使用 hello 调用方法 say_hello --- 三生万物
hello.say_hello() 

print(type(Hello))
print(type(hello))

Hello,world
<class 'type'>
<class '__main__.Hello'>


这就是一个标准的“二生三，三生万物”过程。 从类到我们可以调用的方法，用了这两步。

那我们不由自主要问，类从何而来呢？回到代码的第一行。
class Hello其实是一个函数的“语义化简称”，只为了让代码更浅显易懂，它的另一个写法是：

这样的写法，就和之前的Class Hello写法作用完全相同,建实例并调用,输出效果也一样

In [13]:
# 假如我们有一个函数 fn
def fn(self,name='world'):
    print('Hello,{}'.format(name))

# 通过 type 创建 Hello 类   --- 道生二
Hello = type('Hello',(object,),dict(say_hello=fn))

hello = Hello()

hello.say_hello()

# 换成匿名函数
H = type('H',(object,),dict(say_hello=lambda self,value='world':print('Hello,{}'.format(value))))
h = H()
h.say_hello()

Hello,world
Hello,world


**`Hello = type('Hello', (object,), dict(say_hello=fn)) `**

**type(类名, 父类的元组（针对继承的情况，可以为空），包含属性的字典（名称和值）)**

注意它的三个参数！暗合人类的三大永恒命题：**我是谁，我从哪里来，我要到哪里去.**

第一个参数：我是谁。类名

第二个参数：我从哪里来。继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；

第三个参数：我要到哪里去。需要调用的方法和属性包含到一个字典里，再作为参数传入。class的方法名称与函数绑定，这里我们把函数fn绑定到方法名hello上。


```
class Hello(object){
# class 后声明“我是谁”
# 小括号内声明“我来自哪里”
# 中括号内声明“我要到哪里去”
    def say_hello(){
        
    }
}
```



**以字典的模式给类定义方法或属性**

**属性：**

```
class Foo(object):
    bar = True
```
用type()创建有属性的类(类属性)
```
Foo = type('Foo',(object,),{'bar':Ture})
```

**方法：**

```
class Hello():
    def say_hello(self,name='world'):
        print('Hello,{}'.format(name))
```
用type()创建有方法的类(实例方法)
```
def fn(self,name='world'):
    print('Hello,{}'.format(name))

Hello = type('Hello',(object,),dict(say_hello=fn))
```
换匿名函数
```
Hello = type('Hello',(object,),dict(say_hello=lambda self,name='world':print('Hello,{}'.foramt(name))))
```



造物主，可以直接创造单个的人，但这是一件苦役。造物主会先创造“人”这一物种，再批量创造具体的个人。并将三大永恒命题，一直传递下去。

**“道”可以直接生出“二”，但它会先生出“一”，再批量地制造“二”。**

**type可以直接生成类（class），但也可以先生成元类（metaclass），再使用元类批量定制类（class）。**

## 元类——道生一，一生二

一般来说，元类均被命名后缀为Metalass。想象一下，我们需要一个可以自动打招呼的元类，它里面的类方法呢，有时需要say_Hello，有时需要say_Hi，有时又需要say_Sayolala，有时需要say_Nihao。

如果每个内置的say_xxx都需要在类里面声明一次，那将是多么可怕的苦役！ 不如使用元类来解决问题。

以下是创建一个专门“打招呼”用的元类代码：

In [ ]:
class SayMetaClass(type):
    
    def __new__(cls,name,bases,attrs):
        attrs['say_'+name]=lambda self,value,saying=name:print('{},{}!'.foramt(saying,value))
        return type.__new__(cls,name,bases,attrs)

记住两点：

1、元类是由“type”衍生而出，所以父类需要传入type。**【道生一，所以一必须包含道】**

2、元类的操作都在 \__new__中完成，它的第一个参数是将创建的类，之后的参数即是三大永恒命题：我是谁，我从哪里来，我将到哪里去。 它返回的对象也是三大永恒命题，接下来，这三个参数将一直陪伴我们。

在\__new__中，我只进行了一个操作，就是这个

`attrs['say_'+name] = lambda self,value,saying=name:print('{},{}!'.foramt(saying,value))`

其中匿名函数等同于
```
def niming(self,value,saying=name):
    print('{},{}!'.foramt(saying,value)

```

\__new__()方法接收到的参数依次是：

cls  表示元类

name  表示创建类的类名

bases 表示创建类继承的所有父类

attrs 表示创建类的所有属性和方法（以键值对的字典的形式）




## 根据道生一、一生二、二生三、三生万物的准则，走进元类的生命周期吧！

**注意：通过元类创建的类，第一个参数是父类，第二个参数是metaclass**

In [19]:
# 道生一 传入 type 
class SayMetaClass(type):
    # 传入三大永恒话题： 类名称、父类、属性
    def __new__(cls,name,bases,attrs):
        print(name,bases,attrs)
        attrs['say_'+name] = lambda self,value,saying=name:print('{},{}!'.format(saying,value))
        # 传承三大永恒话题： 类名称、父类、属性
        print(name,bases,attrs)
        return type.__new__(cls,name,bases,attrs)

# 一生二 创建类
class Hello(object,metaclass=SayMetaClass):
    pass

print('==============Hello================')

# 二生三 创建实例
hello = Hello()

# 三生万物 调用实例方法
hello.say_Hello('World')


print('==============Nihao================')

class Nihao(object,metaclass=SayMetaClass):
    pass

nihao = Nihao()
nihao.say_Nihao('中华')

print('===============Sayolala===============')

class Sayolala(object,metaclass=SayMetaClass):
    pass

sayolala = Sayolala()
sayolala.say_Sayolala('Japan')

Hello (<class 'object'>,) {'__module__': '__main__', '__qualname__': 'Hello'}
Hello (<class 'object'>,) {'__module__': '__main__', '__qualname__': 'Hello', 'say_Hello': <function SayMetaClass.__new__.<locals>.<lambda> at 0x00D822B8>}
==============Hello================
Hello,World!
==============Nihao================
Nihao (<class 'object'>,) {'__module__': '__main__', '__qualname__': 'Nihao'}
Nihao (<class 'object'>,) {'__module__': '__main__', '__qualname__': 'Nihao', 'say_Nihao': <function SayMetaClass.__new__.<locals>.<lambda> at 0x00D82E40>}
Nihao,中华!
===============Sayolala===============
Sayolala (<class 'object'>,) {'__module__': '__main__', '__qualname__': 'Sayolala'}
Sayolala (<class 'object'>,) {'__module__': '__main__', '__qualname__': 'Sayolala', 'say_Sayolala': <function SayMetaClass.__new__.<locals>.<lambda> at 0x00D99348>}
Sayolala,Japan!


如果不用元类，要达到上述效果，就需要创建3个类，里面各声明一次，如下：

In [16]:
print('==============Hello================')
class Hello(object):
    def say_Hello(self,value):
        print('Hello,{}!'.format(value))
        
hello = Hello()
hello.say_Hello('World')

print('==============Nihao================')

class Nihao(object):
    def say_Nihao(self,value):
        print('Nihao,{}!'.format(value))
        
nihao = Nihao()
nihao.say_Nihao('中华')

print('===============Sayolala===============')

class Sayolala(object):
    def say_Sayolala(self,value):
        print('Sayolala,{}!'.format(value))
        
sayolala = Sayolala()
sayolala.say_Sayolala('Japan')

==============Hello================
Hello,World!
==============Nihao================
Nihao,中华!
===============Sayolala===============
Sayolala,Japan!


在sayhello的基础上，加上问候方式，轻吻kiss，握手shakehand，鞠躬bow

In [20]:
# 道生一 传入 type 
class SayMetaClass(type):
    # 传入三大永恒话题： 类名称、父类、属性
    def __new__(cls,name,bases,attrs):
        attrs['say_'+name] = lambda self,value,saying=name:print('{},{}!'.format(saying,value))
        attrs['manner'] = lambda self,manner:print('{}！'.format(manner))
        # 传承三大永恒话题： 类名称、父类、属性
        return type.__new__(cls,name,bases,attrs)

# 一生二 创建类
class Hello(object,metaclass=SayMetaClass):
    pass

print('==============Hello================')

# 二生三 创建实例
hello = Hello()

# 三生万物 调用实例方法
hello.say_Hello('World')
hello.manner('Kiss')


print('==============Nihao================')

class Nihao(object,metaclass=SayMetaClass):
    pass

nihao = Nihao()
nihao.say_Nihao('中华')
nihao.manner('Shakehand')

print('===============Sayolala===============')

class Sayolala(object,metaclass=SayMetaClass):
    pass

sayolala = Sayolala()
sayolala.say_Sayolala('Japan')
sayolala.manner('Bow')

==============Hello================
Hello,World!
Kiss！
==============Nihao================
Nihao,中华!
Shakehand！
===============Sayolala===============
Sayolala,Japan!
Bow！


##  给list增加一个list本身没有的add方法

In [23]:
class ListMetaClass(type):
    def __new__(cls,name,bases,attrs):
        attrs['add'] = lambda self,value:self.append(value)
        return type.__new__(cls,name,bases,attrs)

class MyList(list,metaclass=ListMetaClass):
    pass

L = MyList()
L.add(1)
print(L)

L2 = list()
L2.add(1)

[1]


AttributeError: 'list' object has no attribute 'add'

动态修改有什么意义？直接在MyList定义中写上add()方法不是更简单吗？正常情况下，确实应该直接写，通过metaclass修改纯属变态。但是，总会遇到需要通过metaclass修改类定义的。ORM就是一个典型的例子。

## 挑战二：网络代理的爬取


In [29]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'Accept-Encoding': 'gzip, deflate, br', 
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,id;q=0.7'
}

def get_page(url):
    try:
        r = requests.get(url,headers = headers)
        print('Getting result',url,r.status_code)
        if r.status_code == 200:
            return r.text
    except ConnectionError:
        print('Crawling Failed', url)
        return None

if __name__ == '__main__':
    rs = get_page('https://www.baidu.com/')
    print(rs)

Getting result https://www.baidu.com/ 200
<!DOCTYPE html>
<!--STATUS OK-->
























































	
































	
        
			        
	
			        
	
			        
	
			        
			    

	
        
			        
	
			        
	
			        
	
			        
			    
























<html>
<head>
    
    <meta http-equiv="content-type" content="text/html;charset=utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=Edge">
	<meta content="always" name="referrer">
    <meta name="theme-color" content="#2932e1">
    <link rel="shortcut icon" href="/favicon.ico" type="image/x-icon" />
    <link rel="search" type="application/opensearchdescription+xml" href="/content-search.xml" title="ç¾åº¦æç´¢" />
    <link rel="icon" sizes="any" mask href="//www.baidu.com/img/baidu_85beaf5496f291521eb75ba38eacbd87.svg">
	
	
	<link rel="dns-prefetch" href="//s1.bdstatic.com"/>
	<link rel="dns-prefetch" href="//t1.baidu.com"/>
	<link rel="dns-pre

In [37]:
# 道生一 传入 type 
class SayMetaClass(type):
    # 传入三大永恒话题： 类名称、父类、属性
    def __new__(cls,name,bases,attrs):
        print(attrs)
        count = 0
        attrs['__WenhouFunc__'] = []
        attrs['__WenhouName__'] = []
        for k,v in attrs.items():
            if 'Wenhou_' in k:
                attrs['__WenhouFunc__'].append(k)
                attrs['__WenhouName__'].append(v)
                count += 1
        #for k in attrs['__WenhouName__']:
            #attrs.pop(k)
        attrs['__WenhouFuncCount__'] = count
        print(attrs)
        return type.__new__(cls,name,bases,attrs)

# 一生二 创建类
class Hello(object,metaclass=SayMetaClass):
    
    def Wenhou_hello(self):
        pass
    
    def Wenhou_manner(self):
        pass
    
    





'''
class Wenhou_Nihao(object,metaclass=SayMetaClass):
    pass

class Wenhou_Sayolala(object,metaclass=SayMetaClass):
    pass
    
print('==============Hello================')

# 二生三 创建实例
hello = Hello()

# 三生万物 调用实例方法
hello.say_Hello('World')

print('==============Nihao================')



nihao = Nihao()
nihao.say_Nihao('中华')

print('===============Sayolala===============')


sayolala = Sayolala()
sayolala.say_Sayolala('Japan')
'''



{'__module__': '__main__', '__qualname__': 'Hello', 'Wenhou_hello': <function Hello.Wenhou_hello at 0x061A5660>, 'Wenhou_manner': <function Hello.Wenhou_manner at 0x061A5E40>}
{'__module__': '__main__', '__qualname__': 'Hello', 'Wenhou_hello': <function Hello.Wenhou_hello at 0x061A5660>, 'Wenhou_manner': <function Hello.Wenhou_manner at 0x061A5E40>, '__WenhouFunc__': ['Wenhou_hello', 'Wenhou_manner'], '__WenhouName__': [<function Hello.Wenhou_hello at 0x061A5660>, <function Hello.Wenhou_manner at 0x061A5E40>], '__WenhouFuncCount__': 2}


"\nclass Wenhou_Nihao(object,metaclass=SayMetaClass):\n    pass\n\nclass Wenhou_Sayolala(object,metaclass=SayMetaClass):\n    pass\n    \nprint('==============Hello================')\n\n# 二生三 创建实例\nhello = Hello()\n\n# 三生万物 调用实例方法\nhello.say_Hello('World')\n\nprint('==============Nihao================')\n\n\n\nnihao = Nihao()\nnihao.say_Nihao('中华')\n\nprint('===============Sayolala===============')\n\n\nsayolala = Sayolala()\nsayolala.say_Sayolala('Japan')\n"

# python3 下的元类窥探及实现

https://blog.csdn.net/cz19900520/article/details/79641597

### 类也是对象

# Python之Metaclass详解，Python之元类
https://www.cnblogs.com/intimacy/p/8119449.html

## 类同样也是一种对象

In [2]:
class Test(object):
    pass

在只要你使用关键字class，Python解释器在执行的时候就会创建一个对象。

以上代码结束后,python在内存中创建了一个名为 Test 的对象。

这个对象和普通对象有所不同：**拥有创建对象（实例对象）的能力**

但是，它的本质仍然是一个对象，于是乎你可以对它做如下的操作：

- 将他赋值给一个变量
- 拷贝他
- 给他设置属性
- 将他当做方法的参数传递

In [5]:
print(Test) # 你可以打印一个类，因为它其实也是一个对象
print('---------------')
def echo(o):
    print(o)

echo(Test) # 你可以将类做为参数传给函数
print('---------------')
print(hasattr(Test,'a'))
Test.a = 'attribute' # 你可以为类增加属性
print(hasattr(Test,'attribute'))
print(Test.a)
print('---------------')
test = Test # 你可以将类赋值给一个变量
print(test.a)

print('---------------')
print(test()) # 创建实例

<class '__main__.Test'>
---------------
<class '__main__.Test'>
---------------
True
False
attribute
---------------
attribute
---------------


## 动态地创建类

因为**类也是对象，你可以在运行时动态的创建它们，就像其他任何对象一样**。首先，你可以在函数中创建类，使用class关键字即可。

In [13]:
def test(name):
    if name == 'jamie':
        class Jamie(object):
            pass   
        return Jamie   # 返回的是类，不是类的实例
    else:
        class Xujiu(object):
            pass
        return Xujiu
    
my = test('jamie')
print(my)   # 返回的是类，不是类的实例
print(my())   # 你可以通过这个类创建类实例，也就是对象

<class '__main__.test.<locals>.Jamie'>
<__main__.test.<locals>.Jamie object at 0x00CBA690>


但这还不够动态，因为你仍然需要自己编写整个类的代码。由于类也是对象，所以它们必须是通过什么东西来生成的才对。

**当你使用class关键字时，Python解释器自动创建这个对象**。但就和Python中的大多数事情一样，Python仍然提供给你手动处理的方法。

还记得内建函数type吗？这个古老但强大的函数能够让你知道一个对象的类型是什么，就像这样：

In [14]:
print(type(1))   # 数值的类型
print(type('1'))  # 字符串的类型
print(type(my()))   # 实例对象的类型
print(type(my)) # 类的类型

<class 'int'>
<class 'str'>
<class '__main__.test.<locals>.Jamie'>
<class 'type'>


仔细观察上面的运行结果，发现使用type对类 my 查看类型答案为type， 是不是有些惊讶。。。看下面

## 使用type创建类

type还有一种完全不同的功能，动态的创建类。

type可以接受一个类的描述作为参数，然后返回一个类。（要知道，根据传入参数的不同，同一个函数拥有两种完全不同的用法是一件很傻的事情，但这在Python中是为了保持向后兼容性）

type可以像这样工作：

**type(类名, 由父类名称组成的元组（针对继承的情况，可以为空），包含属性的字典（名称和值）)**

注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；

- 用 class 创建类

In [17]:
class Test:  # 定义一个Test类
    pass

Test()  # 创建一个Test类地实例对象

- 用 type 创建类

In [18]:
Test2 = type('Test2',(),{}) 
Test2()

我们使用"Test2"作为类名，并且也可以把它当做一个变量来作为类的引用。类和变量是不同的，这里没有任何理由把事情弄的复杂。即type函数中第1个实参，也可以叫做其他的名字，这个名字表示类的名字

## 使用 type 创建带有属性的类

type 接受一个字典来为类定义属性，因此

In [6]:
Foo = type('Foo',(object,),{'bar': 'value'})
print(Foo.bar)

value


可以翻译为：

In [26]:
class Foo(object):
    bar = 'value'
    def __init__(self):
          self.inst = 'inst value'
            
foo = Foo()
print(Foo.bar)
print(foo.inst)

value
inst value


**注意**

- type的第2个参数，元组中是父类的名字，而不是字符串
-  通过 type 添加的属性是类属性，并不是实例属性，而 class 可以定义类和实例的属性

当然，可以继承这个类

In [29]:
class FooChild(Foo):
    pass

print(FooChild.bar)
foochild = FooChild()
print(foochild.inst)

value
inst value


可以写成为：

In [28]:
FooChild = type('FooChild',(Foo,),{})
print(FooChild.bar)

value


## 使用 type 创建带有方法的类

为类增加方法。只需要定义一个有着恰当签名的函数并将其作为属性赋值就可以了。

**添加实例方法**

函数定义需要self参数

def echo_bar(self):

In [6]:
# 创建带有属性的类
Foo = type('Foo',(object,),{'bar': 'value'})
print(Foo.bar)

# 定义一个普通的函数
def echo_bar(self):
    print(self.bar)
    
# 让FooChild类中的echo_bar属性，指向了上面定义的函数
FooChild = type('FooChild',(Foo,),{'echo_bar':echo_bar})

# 判断 Foo 类种是否有 echo_bar 这个属性
print(hasattr(Foo,'echo_bar'))

# 判断 FooChild 类中 是否有 echo_bar 这个属性
print(hasattr(FooChild,'echo_bar'))

my_foo = FooChild()
my_foo.echo_bar() # 在实例上调用 

value
False
True
value


**添加静态方法**

函数定义可以不传递任何参数

def test_static():

staticmethod 方法的参数是自定义的任何参数，当然也可以没有参数。它不需要一个隐含参数，像self这样的。这是它与实例方法和类方法之间在形式上的一个主要不同之处。

staticmethod即可以在类上调用`C.fun()`也可以在实例上调用`C().fun()`. **在实例上调用时，实例被忽略而类被使用---实际上还是类调用。**


**@staticmethod** means: when this method is called, we don't pass an instance of the class to it (as we normally do with methods). This means you can put a function inside a class but you can't access the instance of that class (this is useful when your method does not use the instance).

classmethod 必须使用类对象作为第一个参数，而 staticmethod 则可以不传递任何参数。

一般类中传递的是 self 类实例

In [9]:
# 创建带有属性的类
Foo = type('Foo',(object,),{'bar': 'value'})

# 定义一个普通的函数
def echo_bar(self):
    print(self.bar)

@staticmethod
def test_static():
    return('static method ...')
    
Foochild = type('Foochild',(Foo,),{'echo_bar':echo_bar,'test_static':test_static})
foochild = Foochild()

print(foochild.test_static)
print(foochild.test_static()) # 在实例上调用方法 (实例被忽略而类被使用---实际上还是类调用)
print(Foochild.test_static())  # 在类上调用方法

<function test_static at 0x058808A0>
static method ...
static method ...


是因为python函数使用return返回值，如果不用
return, 而用print输出值，这个函数默认还有一个返回值为None 

**添加类方法**

函数定义需要cls参数

def test_class(cls)

classmethod需要接收一个隐含参数作为第一个参数，不同于实例方法的self, 类方法的隐含参数叫做cls(其实可以叫任何别的名字，这里主要是因为命名惯例), 表示传入该方法所属的类。 

classmethod即可以在类上调用（C.f()）也可以在实例上调用（C().f()）. **在实例上调用时，实例被忽略而类被使用---实际上还是类调用。**

**@classmethod** means: when this method is called, we pass the class as the first argument instead of the instance of that class (as we normally do with methods). This means you can use the class and its properties inside that method rather than a particular instance.

In [12]:
# 创建带有属性的类
Foo = type('Foo',(object,),{'bar': 'value'})

@classmethod
def test_class(cls):
    return(cls.bar)
    
Testchild = type('Testchild',(Foo,),{"echo_bar":echo_bar, "test_static": test_static, "test_class": test_class})
testchild = Testchild() 
print(Testchild.test_class()) # 在类上调用方法
print(testchild.test_class()) # 在实例上调用方法 (实例被忽略而类被使用---实际上还是类调用)

value
value


## 到底什么是元类

元类就是用来创建类的“东西”。

你创建类就是为了创建类的实例对象，不是吗？但是我们已经学习到了Python中的类也是对象。

**元类就是用来创建这些类（对象）的，元类就是类的类**，你可以这样理解为：

```
Myclass = MetaClass()  # 使用元类创建出一个对象，这个对象称为“类”
my_object = Myclass() # 使用“类”来创建出实例对象
```
你已经看到了type可以让你像这样做：

`myclass = type(类名, 由父类名称组成的元组（针对继承的情况，可以为空），包含属性的字典（名称和值）)`

这是因为函数type实际上是一个元类。**type就是Python在背后用来创建所有类的元类**。现在你想知道那为什么type会全部采用小写形式而不是Type呢？好吧，我猜这是为了和str保持一致性，str是用来创建字符串对象的类，而int是用来创建整数对象的类。type就是创建类对象的类。你可以通过检查__class__属性来看到这一点。**Python中所有的东西，注意，我是指所有的东西——都是对象。这包括整数、字符串、函数以及类。**它们全部都是对象，而且它们都是从一个类创建而来，这个类就是type。


In [14]:
age = 35
print(age.__class__)

name = 'bob'
print(name.__class__)

def foo():pass
print(foo.__class__)

class Bar():pass
b = Bar()
print(b.__class__)

<class 'int'>
<class 'str'>
<class 'function'>
<class '__main__.Bar'>


现在，对于任何一个__class__的__class__属性又是什么呢？

In [15]:
print(age.__class__.__class__)
print(name.__class__.__class__)
print(foo.__class__.__class__)
print(b.__class__.__class__)

<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


因此，元类就是创建类这种对象的东西。type就是Python的内建元类，当然了，你也可以创建自己的元类。

## python3 中的 metaclass

你可以在定义一个类的时候为其添加参数 metaclass。

python3.x的写法
```
class Foo(object,metaclass=something...):
    ...
```

如果你这么做了，Python就会用元类来创建类Foo。小心点，这里面有些技巧。你首先写下class Foo(object)，但是类Foo还没有在内存中创建。Python会在类的定义中寻找__metaclass__属性，如果找到了，Python就会用它来创建类Foo，如果没有找到，就会用内建的type来创建这个类。把下面这段话反复读几次。当你写如下代码时 :

In [ ]:
class Foo(Bar):
    pass

Python做了如下的操作：

- Foo中有__metaclass__这个属性吗？如果是，Python会通过__metaclass__创建一个名字为Foo的类(对象)
- 如果Python没有找到__metaclass__，它会继续在Bar（父类）中寻找__metaclass__属性，并尝试做和前面同样的操作。
- 如果Python在任何父类中都找不到__metaclass__，它就会在模块层次中去寻找__metaclass__，并尝试做同样的操作。
- 如果还是找不到__metaclass__,Python就会用内置的type来创建这个类对象。

现在的问题就是，你可以在__metaclass__中放置些什么代码呢？答案就是：可以创建一个类的东西。那么什么可以用来创建一个类呢？**type，或者任何使用到type或者子类化type的东东都可以**。


## 自定义元类

元类的主要目的就是为了当创建类时能够自动地改变类。

假想一个很傻的例子，你决定在你的模块里所有的类的属性都应该是大写形式。有好几种方法可以办到，但其中一种就是通过在模块级别设定__metaclass__。采用这种方法，这个模块中的所有类都会通过这个元类来创建，我们只需要告诉元类把所有的属性都改成大写形式就万事大吉了。

幸运的是，__metaclass__实际上可以被任意调用，它并不需要是一个正式的类。所以，我们这里就先以一个简单的函数作为例子开始。

In [17]:
def upper_attr(class_name,class_parents,class_attr):
    # 遍历属性字典，把不是__开头的属性名字变为大写
    new_attr = {}
    for name,value in class_attr.items():
        if not name.startswith('__'):
            new_attr[name.upper()] = value
    #调用type来创建一个类    
    return type(class_name, class_parents, new_attr)

class Foo(object,metaclass=upper_attr):
    bar = 'bip'
    
print(hasattr(Foo,'bar'))
print(hasattr(Foo,'BAR'))

print(dir(Foo))

False
True
['BAR', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


现在让我们再做一次，这一次用一个真正的class来当做元类。

- __new__ 是在__init__之前被调用的特殊方法
- __new__是用来创建对象并返回之的方法而 __init__只是用来将传入的参数初始化给对象
- 你很少用到__new__，除非你希望能够控制对象的创建
- 这里，创建的对象是类，我们希望能够自定义它，所以我们这里改写__new__

In [26]:
class Upper_attr(type):
    
    def __new__(cls,name,bases,attrs):
        new_attrs = {}
        for key,value in attrs.items():
            if not key.startswith('__'):
                new_attrs[key.upper()] = value
        
        #复用type.__new__方法
        return type.__new__(cls,name,bases,new_attrs)
    
class Foo(object,metaclass=Upper_attr):
    bar = 'bip'
    
print(hasattr(Foo,'bar'))
print(hasattr(Foo,'BAR'))

print(dir(Foo))

False
True
['BAR', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']
